## Google Books API
The goal of this notebook is to turn the raw information obtained from the google books API into information in the following format:
```
{
  googleId: STRING
  title: STRING
  authors: STRING
  publisher: STRING
  publishedDate: STRING
  description: STRING
  isbn10: STRING
  isbn13: STRING
  pageCount: INTEGER
  categories: STRING
  thumbnail: STRING
  smallThumbnail: STRING
  language: STRING
  webReaderLink: STRING
  textSnippet: STRING
  isEbook: BOOLEAN
}
```

In [0]:
import requests
import json
import pickle
import pandas as pd
import numpy as np
import time

## Obtain sample information from request:

In [0]:
from secrets import GOOGLE_KEY

In [0]:
response = requests.get('https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key='+GOOGLE_KEY)

In [0]:
sample_api_result = json.loads(response.text)

In [0]:
sample_api_result 

{'kind': 'books#volumes',
 'totalItems': 113,
 'items': [{'kind': 'books#volume',
   'id': 'gK98gXR8onwC',
   'etag': 'ym8oWidtoE0',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/gK98gXR8onwC',
   'volumeInfo': {'title': 'Flowers for Algernon',
    'subtitle': 'A One-act Play',
    'authors': ['David Rogers', 'Daniel Keyes'],
    'publisher': 'Dramatic Publishing',
    'publishedDate': '1969',
    'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
     {'type': 'ISBN_13', 'identifier': '9780871293879'}],
    'readingModes': {'text': False, 'image': True},
    'pageCount': 32,
    'printType': 'BOOK',
    'averageRating': 5,
    'ratingsCount': 1,
    'maturityRating': 'NOT_MATURE',
    'allowAnonLogging': False,
    'contentVersion': '0.0.2.0.preview.1',
    'panelizationSummary': {'containsEpubBubbles': False,
     'containsImageBubbles': False},
    'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=fro

### The following is template code to save and load a reference object. 

In [0]:
with open('example_google_books_return.obj','ab') as f:
    pickle.dump(sample_api_result,f)

In [0]:
with open('example_google_books_return.obj','rb') as f:
    test = pickle.load(f)

In [0]:
test

{'kind': 'books#volumes',
 'totalItems': 113,
 'items': [{'kind': 'books#volume',
   'id': 'gK98gXR8onwC',
   'etag': 'ym8oWidtoE0',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/gK98gXR8onwC',
   'volumeInfo': {'title': 'Flowers for Algernon',
    'subtitle': 'A One-act Play',
    'authors': ['David Rogers', 'Daniel Keyes'],
    'publisher': 'Dramatic Publishing',
    'publishedDate': '1969',
    'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
     {'type': 'ISBN_13', 'identifier': '9780871293879'}],
    'readingModes': {'text': False, 'image': True},
    'pageCount': 32,
    'printType': 'BOOK',
    'averageRating': 5,
    'ratingsCount': 1,
    'maturityRating': 'NOT_MATURE',
    'allowAnonLogging': False,
    'contentVersion': '0.0.2.0.preview.1',
    'panelizationSummary': {'containsEpubBubbles': False,
     'containsImageBubbles': False},
    'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=fro

## Extracting necessary information to send to Web

In [0]:
# Results are in ['items'], and by default there are 10 results
print('Top level heirarchy:', sample_api_result.keys(),'\n')
print("type(sample_api_results['items']) = ", type(sample_api_result['items']))
print("len(sample_api_result['items']) = ", len(sample_api_result['items']))

Top level heirarchy: dict_keys(['kind', 'totalItems', 'items']) 

type(sample_api_results['items']) =  <class 'list'>
len(sample_api_result['items']) =  10


In [0]:
# Information stored in one record
sample_api_result['items'][0]

{'kind': 'books#volume',
 'id': 'gK98gXR8onwC',
 'etag': 'ym8oWidtoE0',
 'selfLink': 'https://www.googleapis.com/books/v1/volumes/gK98gXR8onwC',
 'volumeInfo': {'title': 'Flowers for Algernon',
  'subtitle': 'A One-act Play',
  'authors': ['David Rogers', 'Daniel Keyes'],
  'publisher': 'Dramatic Publishing',
  'publishedDate': '1969',
  'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
   {'type': 'ISBN_13', 'identifier': '9780871293879'}],
  'readingModes': {'text': False, 'image': True},
  'pageCount': 32,
  'printType': 'BOOK',
  'averageRating': 5,
  'ratingsCount': 1,
  'maturityRating': 'NOT_MATURE',
  'allowAnonLogging': False,
  'contentVersion': '0.0.2.0.preview.1',
  'panelizationSummary': {'containsEpubBubbles': False,
   'containsImageBubbles': False},
  'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
   'thumbnail': 'http://books.google.com/books/cont

In [0]:
def retreive_details(json_dict,keys_to_extract):
    """
    This is a function that searches through a dictionary, and
    retreives keys that match keys from a list. This function is 
    will find matching keys in every level of the heirarchy.
    
    Inputs:
    json_dict       -  A dictionary expected to be from json.loads() 
                       but any dictionary with JSON-like formatting
                       will do
    keys_to_extract -  A list of keys that you want to extract from the 
                       json_dict object
                       
    Output:
    new_dict        -  A new "flattened"dictionary with all the matching keys. 
                       All the keys are in the top level 
    """
    new_dict={}
    for item in json_dict.keys():
        if type(json_dict[item]) is dict:
            temp_dict = retreive_details(json_dict[item],keys_to_extract)
            new_dict.update(temp_dict)
        if item in keys_to_extract:
            new_dict[item] = json_dict[item] 
    return new_dict

In [0]:
# The details (keys) to extract from the json object
relevant_details=['id','title','authors','publisher',
                  'publishedDate','description','industryIdentifiers',
                  'pageCount','categories','thumbnail','smallThumbnail',
                  'language','webReaderLink','textSnippet','isEbook',
                  'averageRating']

In [0]:
my_new_dictionary = retreive_details(sample_api_result['items'][0],relevant_details)
my_new_dictionary

{'id': 'gK98gXR8onwC',
 'title': 'Flowers for Algernon',
 'authors': ['David Rogers', 'Daniel Keyes'],
 'publisher': 'Dramatic Publishing',
 'publishedDate': '1969',
 'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
  {'type': 'ISBN_13', 'identifier': '9780871293879'}],
 'pageCount': 32,
 'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
 'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
 'language': 'en',
 'isEbook': False,
 'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
 'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINNIAN&nbsp;...'}

In [0]:
# Loop over all the entries in the items list to 
# process all the books into the format that the API endpoint requires

my_dict_list = []
for item in a['items']:
    temp_dict = retreive_details(item,relevant_details)
    my_dict_list.append(temp_dict)


In [0]:
my_dict_list

[{'id': 'gK98gXR8onwC',
  'title': 'Flowers for Algernon',
  'authors': ['David Rogers', 'Daniel Keyes'],
  'publisher': 'Dramatic Publishing',
  'publishedDate': '1969',
  'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
   {'type': 'ISBN_13', 'identifier': '9780871293879'}],
  'pageCount': 32,
  'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
  'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
  'language': 'en',
  'isEbook': False,
  'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
  'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINN

## Final Cleanup of information:
The function defined earlier is generic, and so reusable. The data however still does not perfectly match the data model that the Web team is expecting. The ISBN informaiton needs to be unpacked, and the id key needs to change to googleid

In [0]:
# Starting from cleaning one entry:
a = retreive_details(sample_api_result['items'][0],relevant_details)
a

{'id': 'gK98gXR8onwC',
 'title': 'Flowers for Algernon',
 'authors': ['David Rogers', 'Daniel Keyes'],
 'publisher': 'Dramatic Publishing',
 'publishedDate': '1969',
 'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0871293870'},
  {'type': 'ISBN_13', 'identifier': '9780871293879'}],
 'pageCount': 32,
 'averageRating': 5,
 'ratingsCount': 1,
 'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
 'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
 'language': 'en',
 'isEbook': False,
 'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
 'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his

In [0]:
my_dict = {}
for i in a['industryIdentifiers']:
    my_dict[("".join(i['type'].split('_')).lower())] = i['identifier']

In [0]:
my_dict

{'isbn10': '0871293870', 'isbn13': '9780871293879'}

In [0]:
# Add isbns to dictionary and remove the industryIdentifiers key
a.update(my_dict)
del a['industryIdentifiers']

In [0]:
# Change id to googleId
a['googleId']=a['id']
del a['id']

In [0]:
# final result
a

{'title': 'Flowers for Algernon',
 'authors': ['David Rogers', 'Daniel Keyes'],
 'publisher': 'Dramatic Publishing',
 'publishedDate': '1969',
 'pageCount': 32,
 'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
 'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
 'language': 'en',
 'isEbook': False,
 'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
 'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINNIAN&nbsp;...',
 'isbn10': '0871293870',
 'isbn13': '9780871293879',
 'googleId': 'gK98gXR8onwC'}

An initial concern was the ordering of the dictionary, but whenever any sort of dictionary-like object is accessed it is accessed through a call to the key, so order doesn't matter. Another concern is null values. The Web team has dealt with exception handling, so explicit null values are not required.

In [0]:
def clean(input_dict):
    """
    This function reformats isbns, and modifies the id name
    from a dictionary returned from retreive_details. It also 
    changes the id key to googleId to match the format that the
    API endpoint specifies.
    
    Input:
    input_dict  - a dictionary with only relevant keys.
                -
    """
    
    my_dict = {}
    
    # Change the format of the isbn data
    try:
        for i in input_dict['industryIdentifiers']:
            my_dict[("".join(i['type'].split('_')).lower())] = i['identifier']

        # Add isbns to dictionary and remove the industryIdentifiers key
        input_dict.update(my_dict)
        del input_dict['industryIdentifiers']
    except KeyError:
        pass
    
    # Change id to googleId
    
    # Try/except should not be necessary here.
    # Google Books always returns an id
    input_dict['googleId']=input_dict['id']
    del input_dict['id']
    
    return input_dict

In [0]:
# Testing out cleaning functionality
a = retreive_details(sample_api_result['items'][0],relevant_details)
b = clean(a)
b

{'title': 'Flowers for Algernon',
 'authors': ['David Rogers', 'Daniel Keyes'],
 'publisher': 'Dramatic Publishing',
 'publishedDate': '1969',
 'pageCount': 32,
 'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
 'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
 'language': 'en',
 'isEbook': False,
 'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
 'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINNIAN&nbsp;...',
 'isbn10': '0871293870',
 'isbn13': '9780871293879',
 'googleId': 'gK98gXR8onwC'}

## Putting everything into one function.


In [0]:
def process_list(list_of_entries,keys_to_extract):
    out_list = []
    for i in list_of_entries:
        parsed_dict = retreive_details(i,keys_to_extract)
        clean_dict = clean(parsed_dict)
        out_list.append(clean_dict)
    return out_list

In [0]:
response = requests.get('https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key='+GOOGLE_KEY)

In [0]:
sample_api_result = json.loads(response.text)

In [0]:
process_list(sample_api_result['items'],relevant_details)

[{'title': 'Flowers for Algernon',
  'authors': ['David Rogers', 'Daniel Keyes'],
  'publisher': 'Dramatic Publishing',
  'publishedDate': '1969',
  'pageCount': 32,
  'averageRating': 5,
  'ratingsCount': 1,
  'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
  'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
  'language': 'en',
  'isEbook': False,
  'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
  'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINNIAN&nbsp;...',
  'isbn10': '0871293870',
  'isbn13': '9780871293879',
  'googleId': 'gK98gXR8onwC'},
 {'title': 

### Testing error handling for when the `industryIdentifiers` doesn't exist for an entry

In [0]:
del sample_api_result['items'][0]['volumeInfo']['industryIdentifiers']

In [0]:
# notice the lack of isbn10 and isbn13
clean(retreive_details(sample_api_result['items'][0],relevant_details))

{'title': 'Flowers for Algernon',
 'authors': ['David Rogers', 'Daniel Keyes'],
 'publisher': 'Dramatic Publishing',
 'publishedDate': '1969',
 'pageCount': 32,
 'smallThumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
 'thumbnail': 'http://books.google.com/books/content?id=gK98gXR8onwC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api',
 'language': 'en',
 'isEbook': False,
 'webReaderLink': 'http://play.google.com/books/reader?id=gK98gXR8onwC&hl=&printsec=frontcover&source=gbs_api',
 'textSnippet': '<b>FLOWERS</b> FOR ALGERNON A One-act Play For Four Men And One Woman” <br>\nCHARACTERS DR. STRAUSS . . . . . . . . . . . . . . . . . . . a young neurosurgeon <br>\nPROFESSOR NEMUR . . . . . . . . . . . . . . . . his older colleague ALICE KINNIAN&nbsp;...',
 'googleId': 'gK98gXR8onwC'}

## Testing the module:

This is the minimal code needed to process the request. In the final app the code needs to be 
jsonified, butt this is good enough as proof of concept

In [0]:
import google_books_hf as hf

In [0]:
help(hf)

Help on module google_books_hf:

NAME
    google_books_hf

DESCRIPTION
    This module is a set of helper functions to process data from the google
    books API. The functions are as follows:
    
    retreive_details()  - Retreive specific keys from a dictionary object
    
    clean()             - Alter data to match data model
    
    process_list()      - Process a list of dictionaries to endpoint output format

FUNCTIONS
    clean(input_dict)
        Alter data to match data model 
        
        This function reformats isbns, and modifies the id name
        from a dictionary returned from retreive_details. It also 
        changes the id key to googleId to match the format that the
        API endpoint specifies.
        
        Input:
        input_dict  - a dictionary output from retreive_details using
                      the list of keys required for the BetterReadsDS 
                      API response.
        
        Output:
        output_dict - a dictionary read

In [0]:
from secrets import GOOGLE_KEY
response = requests.get('https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key='+GOOGLE_KEY)
sample_api_result = json.loads(response.text)

relevant_details=['id','title','authors','publisher',
                  'publishedDate','description','industryIdentifiers',
                  'pageCount','categories','thumbnail','smallThumbnail',
                  'language','webReaderLink','textSnippet','isEbook']

In [0]:
process_list(sample_api_result['items'],relevant_details)

NameError: name 'retreive_details' is not defined

## Importing ENV variables instead of secrets.py
This is simply syntax reference because jupyter notebook environment doen't have a .env file like a pipenv does. In the final product this is probably what we are going to be doing because we can load in environment variables to elastic beanstalk
```
import os
GOOGLE_KEY = os.environ['GOOGLE_KEY']
```

## Book ID API endpoint

In [0]:
url = 'https://www.googleapis.com/books/v1/volumes/zyTCAlFPjgYC?key='

In [0]:
from secrets import GOOGLE_KEY

In [0]:
a = requests.get(url+GOOGLE_KEY)

In [0]:
book = json.loads(a.text)

In [0]:
book

{'kind': 'books#volume',
 'id': 'zyTCAlFPjgYC',
 'etag': 'jL6FrE3yeUw',
 'selfLink': 'https://www.googleapis.com/books/v1/volumes/zyTCAlFPjgYC',
 'volumeInfo': {'title': 'The Google Story (2018 Updated Edition)',
  'subtitle': 'Inside the Hottest Business, Media, and Technology Success of Our Time',
  'authors': ['David A. Vise', 'Mark Malseed'],
  'publisher': 'Random House Publishing Group',
  'publishedDate': '2005-11-15',
  'description': '<b> <b>The definitive, bestselling account of the company that changed the way we work and live, updated for the twentieth anniversary of Google’s founding with analysis of its most recent bold moves to redefine the world—and its even more ambitious plans for the future.</b><br></b><br>Moscow-born Sergey Brin and Midwest-born Larry Page dropped out of graduate school at Stanford University to, as they said, “change the world” through a powerful search engine that would organize every bit of information on the Web for free.\xa0<i>The Google Story<

In [0]:
import google_books_hf as hf

In [0]:
relevant_details=['id','title','authors','publisher',
                  'publishedDate','description','industryIdentifiers',
                  'pageCount','categories','thumbnail','smallThumbnail',
                  'language','webReaderLink','textSnippet','isEbook']

In [0]:
details = hf.retreive_details(book, relevant_details)

In [0]:
hf.clean(details)

{'title': 'The Google Story (2018 Updated Edition)',
 'authors': ['David A. Vise', 'Mark Malseed'],
 'publisher': 'Random House Publishing Group',
 'publishedDate': '2005-11-15',
 'description': '<b> <b>The definitive, bestselling account of the company that changed the way we work and live, updated for the twentieth anniversary of Google’s founding with analysis of its most recent bold moves to redefine the world—and its even more ambitious plans for the future.</b><br></b><br>Moscow-born Sergey Brin and Midwest-born Larry Page dropped out of graduate school at Stanford University to, as they said, “change the world” through a powerful search engine that would organize every bit of information on the Web for free.\xa0<i>The Google Story</i>\xa0takes you deep inside the company’s wild ride from an idea that struggled for funding in 1998 to a firm that today rakes in billions in profits. Based on scrupulous research and extraordinary access to Google, this fast-moving narrative reveals 

#Minimum code example
This example retreives results from the google api based on a query string, and formats them. The documentation for the format should be found here. Note that the "clean" function in google_books_hf might have to be edited if the expected data format changes.

In [0]:
# Basic imports
import requests
import json
import pickle

In [0]:
# Google books API key
from secrets import GOOGLE_KEY

In [0]:
# Helper function to return data in Web ready format
from google_books_hf import process_list

In [0]:
# Function to compose queries (added to google_books_hf too)
def gapi_query(query):
    response = requests.get('https://www.googleapis.com/books/v1/volumes?q='
                 + query
                 + '&key='
                 + GOOGLE_KEY)
    result_dict = json.loads(response.text)
    return result_dict['items']

In [0]:
my_query = 'subject:Fantasy'
results = gapi_query(my_query)

In [0]:
relevant_details=['id','title','authors','publisher',
                  'publishedDate','description','industryIdentifiers',
                  'pageCount','categories','thumbnail','smallThumbnail',
                  'language','webReaderLink','textSnippet','isEbook',
                  'averageRating','categories']

In [0]:
process_list(results, relevant_details)